# Using SQLAlchemy

In [1]:
import sqlalchemy as db
import pandas as pd
import os
import csv
import seaborn as sns
import matplotlib.pyplot as plt

## Connecting to database

In [2]:
# connecting to database sql_challenge created in postgresql
engine = db.create_engine('postgresql://localhost:5432/credit-risk')
connection = engine.connect()
metadata = db.MetaData()

In [3]:
#to find all the tables for the database (it should be empty)
engine.table_names()

/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_1935/3558990862.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['train', 'test', 'train_dumy', 'test_dumy']

## used to_sql to insert data 

In [4]:
#For table train
x_train_path=os.path.join('Resources', 'cleaned-data', 'x_train.csv')

with open(x_train_path, 'r') as train:
    x_train = pd.read_csv(train)
x_train.to_sql('train', con=engine, index=False, if_exists='replace')


#For table test
x_test_path=os.path.join('Resources', 'cleaned-data', 'x_test.csv')

with open(x_test_path, 'r') as test:
    x_test = pd.read_csv(test)
x_test.to_sql('test', con=engine, index=False, if_exists='replace')

               #For table train_dumy
x_train_dumy_path=os.path.join('Resources', 'cleaned-data', 'x_train_dumy.csv')

with open(x_train_dumy_path, 'r') as train_dumy:
    x_train_dumy = pd.read_csv(train_dumy)
x_train_dumy.to_sql('train_dumy', con=engine, index=False, if_exists='replace')

#For table test_dumy
x_test_dumy_path=os.path.join('Resources', 'cleaned-data', 'x_test_dumy.csv')

with open(x_test_dumy_path, 'r') as test_dumy:
    x_test_dumy = pd.read_csv(test_dumy)
x_test_dumy.to_sql('test_dumy', con=engine, index=False, if_exists='replace')


In [5]:
#getting the tables from database
test = db.Table('train', metadata, autoload=True, autoload_with=engine)
train = db.Table('test', metadata, autoload=True, autoload_with=engine)
test_dumy = db.Table('train_dumy', metadata, autoload=True, autoload_with=engine)
train_dumy = db.Table('test_dumy', metadata, autoload=True, autoload_with=engine)


In [6]:
# columns of census table
train.c.keys()

['loan_amnt',
 'int_rate',
 'installment',
 'annual_inc',
 'home_ownership',
 'verification_status',
 'pymnt_plan',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 

In [19]:
#adding primary key
connection.execute('alter table train add id serial primary key')
connection.execute('alter table test add id serial primary key')
connection.execute('alter table train_dumy add id serial primary key')
connection.execute('alter table test_dumy add id serial primary key')

# Using SQLAlchemy ORM

# Reflect Tables into SQLAlchemy ORM

In [20]:
# Python SQL toolkit and Object Relational Mapper
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, and_, or_

In [21]:
# create engine to hawaii.sqlite
engine = create_engine('postgresql://localhost:5432/credit-risk')
conn=engine.connect()

In [22]:
# Use the Inspector to explore the database and print the table names
inspector=inspect(engine)
inspector.get_table_names()

['train_dumy', 'train', 'test', 'test_dumy']

In [23]:
# Use Inspector to print the column names and types

print('*'*50)
columns=inspector.get_columns('train')
for column in columns:
    print(column['name'], column['type'])

print('')

print('*'*50)
columns=inspector.get_columns('test')
for column in columns:
    print(column['name'], column['type'])
    


**************************************************
loan_amnt DOUBLE_PRECISION
int_rate DOUBLE_PRECISION
installment DOUBLE_PRECISION
annual_inc DOUBLE_PRECISION
home_ownership TEXT
verification_status TEXT
pymnt_plan TEXT
dti DOUBLE_PRECISION
delinq_2yrs DOUBLE_PRECISION
inq_last_6mths DOUBLE_PRECISION
open_acc DOUBLE_PRECISION
pub_rec DOUBLE_PRECISION
revol_bal DOUBLE_PRECISION
total_acc DOUBLE_PRECISION
initial_list_status TEXT
out_prncp DOUBLE_PRECISION
out_prncp_inv DOUBLE_PRECISION
total_pymnt DOUBLE_PRECISION
total_pymnt_inv DOUBLE_PRECISION
total_rec_prncp DOUBLE_PRECISION
total_rec_int DOUBLE_PRECISION
total_rec_late_fee DOUBLE_PRECISION
recoveries DOUBLE_PRECISION
collection_recovery_fee DOUBLE_PRECISION
last_pymnt_amnt DOUBLE_PRECISION
collections_12_mths_ex_med DOUBLE_PRECISION
policy_code DOUBLE_PRECISION
application_type TEXT
acc_now_delinq DOUBLE_PRECISION
tot_coll_amt DOUBLE_PRECISION
tot_cur_bal DOUBLE_PRECISION
open_acc_6m DOUBLE_PRECISION
open_act_il DOUBLE_PRECISION


In [24]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

In [25]:
# View all of the classes that automap found
Base.classes.keys()

['train_dumy', 'train', 'test', 'test_dumy']

In [26]:
# Save references to each table
Train = Base.classes.train
Test = Base.classes.test
Train_dumy=Base.classes.train_dumy
Test_dumy=Base.classes.test_dumy

In [27]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploring election table

In [28]:
#using pd.read_sql to select and display (also able to see columns name)
df_election=pd.read_sql("SELECT * FROM train", conn)

df_election.head()

,loan_amnt,int_rate,installment,annual_inc,home_ownership,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,id
0,7000.0,0.1894,256.38,75000.0,MORTGAGE,Not Verified,n,28.62,0.0,2.0,...,87.5,0.0,0.0,352260.0,62666.0,35000.0,10000.0,N,N,1
1,40000.0,0.1614,975.71,102000.0,MORTGAGE,Source Verified,n,11.72,2.0,0.0,...,0.0,0.0,0.0,294664.0,109911.0,9000.0,71044.0,N,N,2
2,11000.0,0.2055,294.81,45000.0,RENT,Verified,n,37.25,1.0,3.0,...,7.7,0.0,0.0,92228.0,36007.0,33000.0,46328.0,N,N,3
3,4000.0,0.1612,140.87,38000.0,MORTGAGE,Not Verified,n,42.89,1.0,0.0,...,100.0,0.0,0.0,284273.0,52236.0,13500.0,52017.0,N,N,4
4,14000.0,0.1797,505.93,43000.0,MORTGAGE,Source Verified,n,22.16,1.0,0.0,...,25.0,0.0,0.0,120280.0,88147.0,33300.0,78680.0,N,N,5


# Close session

In [ ]:
# Close Session
session.close()